# Youtube TFT Display Board

### Connect to micropython board

In [7]:
#%serialconnect to --port=/dev/tty.wchusbserial1410 --baud=115200
%serialconnect to --port=/dev/tty.SLAB_USBtoUART --baud=115200

Connecting to --port=/dev/tty.SLAB_USBtoUART --baud=115200 
Ready.


### Run some test commands

In [90]:
print ('hello esp32')

hello esp32


In [17]:
from machine import Pin
from time import sleep

In [47]:
led = Pin(2, Pin.OUT)
led.on()

### Wifi Connection

In [3]:
import network
from config import ssid, password

station = network.WLAN(network.STA_IF)
station.active(True)
station.connect(ssid, password)

### Fetch youtube data

In [506]:
from urequests import get

In [5]:
from config import api_key, channel_id

base_url = 'https://www.googleapis.com/youtube/v3'

In [508]:
channel_data_url = base_url + '/channels?id=' + channel_id + '&part=statistics&key=' + api_key
print(channel_data_url)
res = get(channel_data_url, stream=True)

https://www.googleapis.com/youtube/v3/channels?id=UC89Gzw1wIFXganAibMAnCOg&part=statistics&key=REDACTED


In [509]:
json_data = res.json()
items = json_data['items']
channel_data = items[0]
stats = channel_data['statistics']
print(stats)

{'videoCount': '15', 'subscriberCount': '395', 'viewCount': '16997', 'hiddenSubscriberCount': False, 'commentCount': '0'}


In [510]:
video_count = stats['videoCount']
sub_count = stats['subscriberCount']
view_count = stats['viewCount']
print(video_count,sub_count,view_count)

15 395 16997


In [517]:
import gc

def get_recent_videos(channel_id,results=10):
    search_url = base_url + '/search?part=snippet&type=video'
    search_url += '&channelId=' + channel_id
    search_url += '&maxResults=' + str(results)
    search_url += '&order=viewCount'
    search_url += '&publishedAfter=2019-01-01T00:00:00Z'
    search_url += '&key=' + api_key
    print(search_url)
    res = get(search_url, stream=True)
    json_data = res.json()
    items = json_data['items']
    return items

print(gc.mem_free())
gc.collect()    
print(gc.mem_free())
videos = get_recent_videos(channel_id)

84944
87728
https://www.googleapis.com/youtube/v3/search?part=snippet&type=video&channelId=UC89Gzw1wIFXganAibMAnCOg&maxResults=10&order=viewCount&publishedAfter=2019-01-01T00:00:00Z&key=REDACTED
.

In [518]:
videos = map(lambda video: { "id": video['id']['videoId'], "title": video['snippet']['title'] }, videos)
videos = list(videos)
print(videos)

[{'id': 'SY8xaJotS7M', 'title': 'Tour na Minha Casa Inteligente 2019'}, {'id': 'FU8GYvFz-QU', 'title': 'Novidades IoT #5 - Internet dos Pombos, ESP32 Telefone, Cafeteira do futuro e Camera por voz'}, {'id': '5aahcavra0E', 'title': 'Novidades IoT #1 - NFC, Google Cloud IoT Device SDK,  Blockchain + IoT e Campus Party'}, {'id': '39E8qGwYr0g', 'title': 'Novidades IoT #2 - TensorFlow em Microcontroladores, Google Coral, Ethereum e Hamsters saud\xe1veis'}, {'id': '2wgBHPRbB8g', 'title': 'Novidades IoT #4 - NVIDIA Jetson, Vis\xe3o Computacional, Automa\xe7\xe3o Residencial e conex\xe3o com a Nuvem'}, {'id': 'gYWJMSNDD_0', 'title': 'Novidades IoT #3 - Impress\xe3o 3d com IA, Sony e Raspberry Pi, Jogos no ESP8266 e formato Feather'}]


In [529]:
def get_video_stat(video_id):
    video_url = base_url + '/videos?part=statistics'
    video_url += '&id=' + video_id
    video_url += '&key=' + api_key
    print(video_url)
    res = get(video_url, stream=True)
    json_data = res.json()
    items = json_data['items']
    data = items[0]
    stat = data['statistics']
    return stat

for video in videos:
    stat = get_video_stat(video['id'])
    video['stat'] = stat    

https://www.googleapis.com/youtube/v3/videos?part=statistics&id=SY8xaJotS7M&key=REDACTED
https://www.googleapis.com/youtube/v3/videos?part=statistics&id=FU8GYvFz-QU&key=REDACTED
https://www.googleapis.com/youtube/v3/videos?part=statistics&id=5aahcavra0E&key=REDACTED
https://www.googleapis.com/youtube/v3/videos?part=statistics&id=39E8qGwYr0g&key=REDACTED
.https://www.googleapis.com/youtube/v3/videos?part=statistics&id=2wgBHPRbB8g&key=REDACTED
https://www.googleapis.com/youtube/v3/videos?part=statistics&id=gYWJMSNDD_0&key=REDACTED


In [426]:
print(videos)

[{'id': 'SY8xaJotS7M', 'title': 'Tour na Minha Casa Inteligente 2019', 'stat': {'commentCount': '26', 'dislikeCount': '0', 'favoriteCount': '0', 'viewCount': '1130', 'likeCount': '165'}}, {'id': 'FU8GYvFz-QU', 'title': 'Novidades IoT #5 - Internet dos Pombos, ESP32 Telefone, Cafeteira do futuro e Camera por voz', 'stat': {'commentCount': '3', 'dislikeCount': '0', 'favoriteCount': '0', 'viewCount': '259', 'likeCount': '36'}}, {'id': '5aahcavra0E', 'title': 'Novidades IoT #1 - NFC, Google Cloud IoT Device SDK,  Blockchain + IoT e Campus Party', 'stat': {'commentCount': '16', 'dislikeCount': '0', 'favoriteCount': '0', 'viewCount': '209', 'likeCount': '42'}}, {'id': '39E8qGwYr0g', 'title': 'Novidades IoT #2 - TensorFlow em Microcontroladores, Google Coral, Ethereum e Hamsters saud\xe1veis', 'stat': {'commentCount': '6', 'dislikeCount': '1', 'favoriteCount': '0', 'viewCount': '188', 'likeCount': '26'}}, {'id': '2wgBHPRbB8g', 'title': 'Novidades IoT #4 - NVIDIA Jetson, Vis\xe3o Computacional

In [519]:
print(gc.mem_free())
gc.collect()    
print(gc.mem_free())

65856
89408


### TFT Display

In [520]:
import st7735
from rgb import color565
from machine import SPI,Pin
import machine

In [521]:
spi = SPI(-1,baudrate=64000000, sck=Pin(26), miso=Pin(5), mosi=Pin(22))
cs = Pin(16) # D8
dc = Pin(21) # D3
rst = Pin(17) # D4

w = 128
h = 160
x = 0
y = 0

display = st7735.ST7735R(spi, dc=dc,cs=cs,rst=rst)

In [522]:
display.fill(color565(255,255,255))

In [527]:
import framebuf
import gc
buf = bytearray(w * h * 2)
fb = framebuf.FrameBuffer(buf, w, h, framebuf.RGB565)
gc.collect()

In [530]:
white = color565(255,255,255)
red = color565(255,0,0)
black = color565(0,0,0)
fb.fill(white)

fb.fill_rect(0,0,w,16,red)
fb.text('Youtube Stats',10,5,white)
fb.hline(0, 16, h, white)

fb.fill_rect(0,17,w,27,black)
fb.text(str(view_count) + ' Views',4,22,white)
fb.text(str(sub_count) + ' Subscribers',4,32,white)
#fb.text(str(video_count) + ' Videos',4,50,red)
fb.hline(0, 44, h, red)

#print(videos)
# 'stat': {'commentCount': '5', 'dislikeCount': '0', 'favoriteCount': '0', 'viewCount': '104', 'likeCount': '21'}}]
# print(videos[0]['title'])
i = 0
for video in videos:
    fb.text(video['title'],4,48 + i*20,black)
    fb.text(video['stat']['viewCount'] + ' | ' + video['stat']['likeCount'] ,4,58 + i*20,red)
    i+=1

display.blit_buffer(buf, x, y, w, h)
gc.collect()